### [evaluate-metric](https://huggingface.co/evaluate-metric)

https://huggingface.co/spaces/evaluate-metric/bleu

https://huggingface.co/spaces/evaluate-metric/meteor

https://huggingface.co/spaces/evaluate-metric/rouge

https://huggingface.co/spaces/evaluate-metric/perplexity